In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.random.set_seed(42)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:  tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e: print(e)

from tensorflow import keras
from keras.utils import plot_model
from tensorflow.keras import layers, models, datasets, utils
from tensorflow.keras.layers import Conv2D, Dense, Dropout, BatchNormalization, MaxPooling2D, Concatenate, Input, GlobalAveragePooling2D
import numpy as np

from utils.model_utils import *

In [11]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Normalize the images to a range of 0-1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

print(f"Shape of x_train: {x_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of x_test: {x_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of x_train: (50000, 32, 32, 3)
Shape of y_train: (50000, 10)
Shape of x_test: (10000, 32, 32, 3)
Shape of y_test: (10000, 10)


In [39]:
def inception_module_naive(input_tensor, filters_1x1, filters_3x3, filters_5x5, idx_inception_block):
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', name=f"prunable_conv_{1+idx_inception_block*3}")(input_tensor)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', name=f"prunable_conv_{1+idx_inception_block*3+1}")(input_tensor)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', name=f"prunable_conv_{1+idx_inception_block*3+2}")(input_tensor)
    max_pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
    
    output = Concatenate(axis=-1)([conv_1x1, conv_3x3, conv_5x5, max_pool_proj])
    return output

def my_inception_module_naive(input_tensor, filters_1x1, filters_3x3, filters_5x5, idx_inception_block):
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', name=f"prunable_conv_{1+idx_inception_block*3}")(input_tensor)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', name=f"prunable_conv_{1+idx_inception_block*3+1}")(input_tensor)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', name=f"prunable_conv_{1+idx_inception_block*3+2}")(input_tensor)
    
    output = Concatenate(axis=-1)([conv_1x1, conv_3x3, conv_5x5])
    return output

def define_inception_model(input_shape, output_shape, list_number_filters=[64, 128, 32]):
    input_tensor = Input(shape=input_shape)

    X = Conv2D(filters=32, kernel_size=3, strides=2, activation='relu', name=f'prunable_conv_0')(input_tensor)

    for idx_inception_block in range(0, len(list_number_filters), 3):
        list_current_filters = list_number_filters[idx_inception_block:idx_inception_block+3]
        filters_1x1, filters_3x3, filters_5x5 = list_current_filters
        X = my_inception_module_naive(X, filters_1x1, filters_3x3, filters_5x5, idx_inception_block)
        X = BatchNormalization()(X)

    X = Conv2D(filters=96, kernel_size=3, strides=2, activation='relu', name=f'prunable_conv_6')(X)
    X = Conv2D(filters=96, kernel_size=3, strides=2, activation='relu', name=f'prunable_conv_7')(X)

    # X = Flatten()(X)
    X = GlobalAveragePooling2D()(X)

    X = Dense(50, activation='relu')(X)
    X = Dense(output_shape, activation='softmax')(X)

    model = tf.keras.Model(inputs=input_tensor, outputs=X)
    return model

In [40]:
keras.backend.clear_session()
LIST_NUMBER_FILTERS = [16, 16, 16, 32, 32, 32]
model = define_inception_model((32, 32, 3), 10, LIST_NUMBER_FILTERS)
# model = Define_Simple_CNN_Model((32, 32, 3), 10, list_number_filters=[32, 64])

print(f"Number of params: {model.count_params()}")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
plot_model(model, show_shapes=True, show_layer_names=True);

Number of params: 244160


In [41]:
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
625/625 [==============================] - 2s 2ms/step - loss: 1.8098 - accuracy: 0.3107 - val_loss: 1.5561 - val_accuracy: 0.4170
Epoch 2/20
625/625 [==============================] - 1s 2ms/step - loss: 1.4371 - accuracy: 0.4697 - val_loss: 1.3058 - val_accuracy: 0.5240
Epoch 3/20
625/625 [==============================] - 1s 2ms/step - loss: 1.2452 - accuracy: 0.5492 - val_loss: 1.1619 - val_accuracy: 0.5764
Epoch 4/20
625/625 [==============================] - 1s 2ms/step - loss: 1.1241 - accuracy: 0.5953 - val_loss: 1.0740 - val_accuracy: 0.6123
Epoch 5/20
625/625 [==============================] - 1s 2ms/step - loss: 1.0352 - accuracy: 0.6273 - val_loss: 1.0076 - val_accuracy: 0.6458
Epoch 6/20
625/625 [==============================] - 1s 2ms/step - loss: 0.9566 - accuracy: 0.6578 - val_loss: 1.0122 - val_accuracy: 0.6402
Epoch 7/20
625/625 [==============================] - 1s 2ms/step - loss: 0.8929 - accuracy: 0.6809 - val_loss: 0.9282 - val_accuracy: 0.6698
Epoch 

In [42]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy:.4f}')

313/313 - 0s - loss: 1.0802 - accuracy: 0.7068 - 378ms/epoch - 1ms/step
Test accuracy: 0.7068
